# **Requirements Installation**

In [ ]:
!pip install datasets twint vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.5/477.5 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/6

# **Import Libraries**

In [ ]:
import pandas as pd
from datasets import load_dataset
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import twint
import random
import re
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import numpy as np
import re
import requests

In [ ]:
dataset = load_dataset("edarchimbaud/news-stocks")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/22025 [00:00<?, ? examples/s]

In [ ]:
df = dataset['train'].to_pandas()
df

,symbol,body,publisher,publish_time,title,url,uuid
0,A,Vipshop Holdings Limited VIPS is set to report...,Zacks,2023-05-18 16:14:04+00:00,Vipshop (VIPS) to Post Q1 Earnings: What's in ...,https://finance.yahoo.com/news/vipshop-vips-po...,27293957-38d0-3710-8e36-c33a88428202
1,A,"SANTA CLARA, Calif., May 19, 2023--(BUSINESS W...",Business Wire,2023-05-19 01:25:00+00:00,Agilent to Appeal Patent Office Decision on CR...,https://finance.yahoo.com/news/agilent-appeal-...,5477d2fd-f0e0-3721-aeb5-9e4184af5fc1
2,A,Agilent Technologies A is set to report its se...,Zacks,2023-05-19 15:08:03+00:00,Agilent Technologies (A) to Post Q2 Earnings: ...,https://finance.yahoo.com/news/agilent-technol...,619f9fb9-c102-37fc-8b8c-50c11237b9a7
3,A,"Earnings reports from Zoom Video, Lowe’s, Snow...",Barrons.com,2023-05-21 19:00:00+00:00,"Costco, JPMorgan, Snowflake, Ford, Zoom, and M...",https://finance.yahoo.com/m/5f7a781e-1e0c-30b0...,5f7a781e-1e0c-30b0-a052-1d88fc0ce184
4,A,Agilent Technologies A reported second-quarter...,Zacks,2023-05-24 17:17:00+00:00,"Agilent (A) Q2 Earnings Match Estimates, Reven...",https://finance.yahoo.com/news/agilent-q2-earn...,e9307f15-4308-33d8-a649-57f4df3530b1
...,...,...,...,...,...,...,...
22020,ZTS,When considering what names to put on your wat...,Investor's Business Daily,2023-08-09 18:18:00+00:00,Drugmaker Zoetis Stock Shows Rising Relative S...,https://finance.yahoo.com/m/de58f2c0-d31a-310c...,de58f2c0-d31a-310c-a293-02e6c4206e8c
22021,ZTS,"LINCOLN, Neb., August 09, 2023--(BUSINESS WIRE...",Business Wire,2023-08-09 22:33:00+00:00,Zoetis Welcomes Officials to Open New State-of...,https://finance.yahoo.com/news/zoetis-welcomes...,51662184-e4c7-3e7f-a6a2-7b0666de92cd
22022,ZTS,Investors interested in stocks from the Medica...,Zacks,2023-08-10 15:40:11+00:00,USNA or ZTS: Which Is the Better Value Stock R...,https://finance.yahoo.com/news/usna-zts-better...,14a5e82f-6510-30a5-bbf1-cc4982348c42
22023,ZTS,"In this article, we will be taking a look at t...",Insider Monkey,2023-11-09 08:49:40+00:00,Top 20 Drug Companies in the US by Revenue,https://finance.yahoo.com/news/top-20-drug-com...,636f3ccc-872c-3532-ad4b-675b269c4602


In [ ]:
df.isna().count()

symbol          22025
body            22025
publisher       22025
publish_time    22025
title           22025
url             22025
uuid            22025
dtype: int64

In [ ]:
df = df.drop(['publisher', 'url', 'uuid'], axis=1)

In [ ]:
print(list(df['symbol'].unique()))

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABEO', 'ABNB', 'ABT', 'ABVC', 'AC', 'ACCO', 'ACGL', 'ACHC', 'ACLS', 'ACMR', 'ACN', 'ACNT', 'ACR', 'ACRE', 'ACRS', 'ACRX', 'ADBE', 'ADC', 'ADES', 'ADI', 'ADIL', 'ADM', 'ADMA', 'ADP', 'ADPT', 'ADSK', 'ADT', 'ADTN', 'ADV', 'AE', 'AEE', 'AEF', 'AEMD', 'AEP', 'AES', 'AEY', 'AEYE', 'AFG', 'AFL', 'AGCO', 'AGL', 'AGNC', 'AGX', 'AHH', 'AIG', 'AIRI', 'AIRT', 'AIT', 'AIZ', 'AJG', 'AKAM', 'AKLI', 'ALB', 'ALEC', 'ALG', 'ALGM', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALLO', 'ALLY', 'ALPN', 'ALRN', 'ALTI', 'ALV', 'ALX', 'AM', 'AMAT', 'AMBC', 'AMC', 'AMCR', 'AMCX', 'AMD', 'AME', 'AMGN', 'AMKR', 'AMP', 'AMRK', 'AMST', 'AMT', 'AMTB', 'AMZN', 'AN', 'ANEB', 'ANET', 'ANSS', 'AON', 'AORT', 'AOS', 'AP', 'APA', 'APCX', 'APD', 'APH', 'APLD', 'APLT', 'APPF', 'APPS', 'APTV', 'AQMS', 'AR', 'ARCT', 'ARDX', 'ARE', 'ARIS', 'ARKO', 'ARKR', 'ARLO', 'ARQT', 'ARTL', 'ARW', 'ASLE', 'ASMB', 'ASPI', 'ASPS', 'ASR', 'ASRV', 'ASTR', 'ASTS', 'ASXC', 'ATEX', 'ATHX', 'ATI', 'ATKR', 'ATNI', 'ATO

In [ ]:
df['publish_time']

0       2023-05-18 16:14:04+00:00
1       2023-05-19 01:25:00+00:00
2       2023-05-19 15:08:03+00:00
3       2023-05-21 19:00:00+00:00
4       2023-05-24 17:17:00+00:00
                   ...           
22020   2023-08-09 18:18:00+00:00
22021   2023-08-09 22:33:00+00:00
22022   2023-08-10 15:40:11+00:00
22023   2023-11-09 08:49:40+00:00
22024   2023-11-12 11:00:55+00:00
Name: publish_time, Length: 22025, dtype: datetime64[ns, GMT]

In [ ]:
# Convert 'publish_time' column to datetime format
df['publish_time'] = pd.to_datetime(df['publish_time'])

# Extract year, month, and day part
df['publish_time'] = df['publish_time'].dt.date

In [ ]:
df

,symbol,body,publish_time,title
0,A,Vipshop Holdings Limited VIPS is set to report...,2023-05-18,Vipshop (VIPS) to Post Q1 Earnings: What's in ...
1,A,"SANTA CLARA, Calif., May 19, 2023--(BUSINESS W...",2023-05-19,Agilent to Appeal Patent Office Decision on CR...
2,A,Agilent Technologies A is set to report its se...,2023-05-19,Agilent Technologies (A) to Post Q2 Earnings: ...
3,A,"Earnings reports from Zoom Video, Lowe’s, Snow...",2023-05-21,"Costco, JPMorgan, Snowflake, Ford, Zoom, and M..."
4,A,Agilent Technologies A reported second-quarter...,2023-05-24,"Agilent (A) Q2 Earnings Match Estimates, Reven..."
...,...,...,...,...
22020,ZTS,When considering what names to put on your wat...,2023-08-09,Drugmaker Zoetis Stock Shows Rising Relative S...
22021,ZTS,"LINCOLN, Neb., August 09, 2023--(BUSINESS WIRE...",2023-08-09,Zoetis Welcomes Officials to Open New State-of...
22022,ZTS,Investors interested in stocks from the Medica...,2023-08-10,USNA or ZTS: Which Is the Better Value Stock R...
22023,ZTS,"In this article, we will be taking a look at t...",2023-11-09,Top 20 Drug Companies in the US by Revenue


# **Data Cleaning And Processing**



1.   Remove all links starting with http or https.
2.   Remove all special characters and emoticons.
1.   Remove all hashtags (#) and @ symbols.
2.   Remove all numbers.

In [ ]:
randomlist = random.sample(range(0, len(df)), 100)

for i in df.itertuples():
    if i[0] in randomlist:
        print(f"Sentiment No. {i[0]}: {i[2]}")
    else:
        pass

Sentiment No. 68: American Airlines flyers purchasing basic economy tickets will no longer get a free checked bag on transoceanic routes. Find out what fees you'll pay.Continue reading
Sentiment No. 604: In this article, we will be looking at 10 stocks Cramer thinks are climbing in this market. To explore similar stocks, you can take a look at 5 Stocks Jim Cramer Thinks Are Climbing In This Market."The Winners Make No Sense"On May 9, CNBC's Mad Money host, Jim Cramer, dived into the current market dynamics to try and unravel the perplexing trends and developments we've been seeing unfold in the market so far in 2023. While bears have been whispering that a "financial storm" has been brewing and has "snuck up on us," with a debt ceiling crisis and economic recession on the horizon, Cramer notes that the market still has not "cratered." Everyone seems to be confused about how the market has been "hanging in there." To deal with this confusion, Cramer went through several financial chart 

In [ ]:
def decontracted(phrase):
    # Specific contractions
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # General contractions
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'https?://\S+', ' ', text)  # Remove URLs
    text = re.sub(r'\#\w+', '', text)  # Remove hashtags
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters (excluding alphanumeric and whitespace)
    text = re.sub(r'_+', ' ', text)  # Replace underscores with space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = decontracted(text)  # Decontract words
    return text.strip()  # Trim leading/trailing spaces

In [ ]:
for i in tqdm(df.itertuples()):
    df.at[i[0], 'processed_data'] = clean(i[2])

22025it [00:21, 1017.36it/s]


In [ ]:
df['processed_data']

0        vipshop holdings limited vips is set to report...
1        santa clara calif may business wireagilent tec...
2        agilent technologies a is set to report its se...
3        earnings reports from zoom video lowes snowfla...
4        agilent technologies a reported secondquarter ...
                               ...                        
22020    when considering what names to put on your wat...
22021    lincoln neb august business wirezoetisthis pre...
22022    investors interested in stocks from the medica...
22023    in this article we will be taking a look at th...
22024    key insightsusing the stage free cash flow to ...
Name: processed_data, Length: 22025, dtype: object

# **Combining the same date news**

In [ ]:
df.columns

Index(['symbol', 'body', 'publish_time', 'title', 'processed_data'], dtype='object')

In [ ]:
df_processed = df.drop(["body", "title"], axis=1)
df_processed

,symbol,publish_time,processed_data
0,A,2023-05-18,vipshop holdings limited vips is set to report...
1,A,2023-05-19,santa clara calif may business wireagilent tec...
2,A,2023-05-19,agilent technologies a is set to report its se...
3,A,2023-05-21,earnings reports from zoom video lowes snowfla...
4,A,2023-05-24,agilent technologies a reported secondquarter ...
...,...,...,...
22020,ZTS,2023-08-09,when considering what names to put on your wat...
22021,ZTS,2023-08-09,lincoln neb august business wirezoetisthis pre...
22022,ZTS,2023-08-10,investors interested in stocks from the medica...
22023,ZTS,2023-11-09,in this article we will be taking a look at th...


In [ ]:
df_processed['news_data_combined'] = df_processed.groupby(['symbol', 'publish_time'])['processed_data'].transform(lambda x: ' '.join(x))
df_processed

,symbol,publish_time,processed_data,news_data_combined
0,A,2023-05-18,vipshop holdings limited vips is set to report...,vipshop holdings limited vips is set to report...
1,A,2023-05-19,santa clara calif may business wireagilent tec...,santa clara calif may business wireagilent tec...
2,A,2023-05-19,agilent technologies a is set to report its se...,santa clara calif may business wireagilent tec...
3,A,2023-05-21,earnings reports from zoom video lowes snowfla...,earnings reports from zoom video lowes snowfla...
4,A,2023-05-24,agilent technologies a reported secondquarter ...,agilent technologies a reported secondquarter ...
...,...,...,...,...
22020,ZTS,2023-08-09,when considering what names to put on your wat...,when considering what names to put on your wat...
22021,ZTS,2023-08-09,lincoln neb august business wirezoetisthis pre...,when considering what names to put on your wat...
22022,ZTS,2023-08-10,investors interested in stocks from the medica...,investors interested in stocks from the medica...
22023,ZTS,2023-11-09,in this article we will be taking a look at th...,in this article we will be taking a look at th...


In [ ]:
df_news = df_processed.drop(['processed_data'], axis=1)
df_news

,symbol,publish_time,news_data_combined
0,A,2023-05-18,vipshop holdings limited vips is set to report...
1,A,2023-05-19,santa clara calif may business wireagilent tec...
2,A,2023-05-19,santa clara calif may business wireagilent tec...
3,A,2023-05-21,earnings reports from zoom video lowes snowfla...
4,A,2023-05-24,agilent technologies a reported secondquarter ...
...,...,...,...
22020,ZTS,2023-08-09,when considering what names to put on your wat...
22021,ZTS,2023-08-09,when considering what names to put on your wat...
22022,ZTS,2023-08-10,investors interested in stocks from the medica...
22023,ZTS,2023-11-09,in this article we will be taking a look at th...


In [ ]:
df_news = df_news.drop_duplicates()
df_news

,symbol,publish_time,news_data_combined
0,A,2023-05-18,vipshop holdings limited vips is set to report...
1,A,2023-05-19,santa clara calif may business wireagilent tec...
3,A,2023-05-21,earnings reports from zoom video lowes snowfla...
4,A,2023-05-24,agilent technologies a reported secondquarter ...
7,A,2023-05-26,in this article we discuss growth stocks to wa...
...,...,...,...
22018,ZTS,2023-08-08,hosting the call today is steve frank vice pre...
22020,ZTS,2023-08-09,when considering what names to put on your wat...
22022,ZTS,2023-08-10,investors interested in stocks from the medica...
22023,ZTS,2023-11-09,in this article we will be taking a look at th...


# **Get the Stock Data**

In [ ]:
import yfinance as yf

# List of stock symbols
symbols = list(df_news['symbol'].unique())
not_in_yfinance = []  # Updated variable name to reflect the context

def get_stock_data(symbol):
    try:
        data = yf.download(symbol)
        return data
    except Exception as e:
        # Check if the error is specifically because the symbol was not found
        if str(e).startswith("No data found"):
            print(f"Symbol '{symbol}' not found in yfinance.")
            not_in_yfinance.append(symbol)  # Add the symbol to the list of those not found
        else:
            raise e  # Re-raise the exception if it's something else

# Fetching data for each symbol and storing in a dictionary
stock_data = {}
for symbol in symbols:
    stock_data[symbol] = get_stock_data(symbol)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


KeyboardInterrupt: 

In [ ]:
not_in_yfinance = ['ABC', 'ACRX', 'ADES', 'AEY', 'ATHX', 'ATVI', 'BF.B', 'BRK.B', 'BVH', 'CDAY', 'CHS', 'DUNE', 'EAR', 'MDWT', 'NFTG', 'PCTI', 'PEAK', 'PNT', 'RPT', 'SCU', 'SRC', 'TMPO', 'WMC']
df_news = df_news[~df_news['symbol'].isin(not_in_yfinance)]
df_news

,symbol,publish_time,news_data_combined
0,A,2023-05-18,vipshop holdings limited vips is set to report...
1,A,2023-05-19,santa clara calif may business wireagilent tec...
3,A,2023-05-21,earnings reports from zoom video lowes snowfla...
4,A,2023-05-24,agilent technologies a reported secondquarter ...
7,A,2023-05-26,in this article we discuss growth stocks to wa...
...,...,...,...
22018,ZTS,2023-08-08,hosting the call today is steve frank vice pre...
22020,ZTS,2023-08-09,when considering what names to put on your wat...
22022,ZTS,2023-08-10,investors interested in stocks from the medica...
22023,ZTS,2023-11-09,in this article we will be taking a look at th...


In [ ]:
stock_data['A']

In [ ]:
for symbol, data in stock_data.items():
  data.reset_index(inplace=True)
  data.rename(columns={'Date': 'publish_time'}, inplace=True)
  data['publish_time'] = pd.to_datetime(data['publish_time'])
  data['publish_time'] = data['publish_time'].dt.date
  data['symbol']=symbol
  stock_data[symbol]=data

In [ ]:
print(type(df_news['publish_time'][1]))
# print(type(stock_data['A']['publish_time'][1]))

In [ ]:
keys = list(stock_data.keys())

result = pd.concat([stock_data[keys[0]], stock_data[keys[1]]], axis=0)

for i in range(2, len(keys)):
  result = pd.concat([result, stock_data[keys[i]]], axis=0)
result

In [ ]:
result.to_csv('result.csv')

In [ ]:
df = pd.read_csv('result.csv')

In [ ]:
df_news[df_news['publish_time']=='2023-05-26']

,symbol,publish_time,news_data_combined


# **Merge the news and stocks opening and closing price**

In [ ]:
df1 = df_news
df2 = df

merged_df = pd.DataFrame()

# Convert 'publish_time' columns to datetime if they are not already in datetime format
if not pd.api.types.is_datetime64_any_dtype(df1['publish_time']):
    df1['publish_time'] = pd.to_datetime(df1['publish_time'])

if not pd.api.types.is_datetime64_any_dtype(df2['publish_time']):
    df2['publish_time'] = pd.to_datetime(df2['publish_time'])

# Iterate through each row in df1
for index, row in df1.iterrows():
    # Filter df2 to include only rows with the same symbol as the current row in df1
    df2_filtered = df2[df2['symbol'] == row['symbol']]
    # Find the closest date in df2_filtered to the current date in df1
    closest_date = df2_filtered.loc[(df2_filtered['publish_time'] - row['publish_time']).abs().idxmin()]

    # Merge the rows from df1 and df2 based on the closest date
    merged_row = pd.concat([row, closest_date], axis=0)
    merged_df = pd.concat([merged_df, merged_row], axis=1)

merged_df

ValueError: attempt to get argmin of an empty sequence

In [ ]:
merged_df

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
symbol,A,A,A,A,A,A,A,A,A,A,...,INCY,INCY,INCY,INCY,INFA,INFA,INFN,INFN,ING,ING
publish_time,2023-05-18 00:00:00,2023-05-19 00:00:00,2023-05-21 00:00:00,2023-05-24 00:00:00,2023-05-26 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00,2023-06-06 00:00:00,2023-06-07 00:00:00,2023-06-08 00:00:00,...,2023-08-04 00:00:00,2023-08-08 00:00:00,2023-11-12 00:00:00,2023-11-14 00:00:00,2023-11-18 00:00:00,2023-11-19 00:00:00,2023-11-09 00:00:00,2023-11-16 00:00:00,2023-11-14 00:00:00,2023-11-17 00:00:00
news_data_combined,vipshop holdings limited vips is set to report...,santa clara calif may business wireagilent tec...,earnings reports from zoom video lowes snowfla...,agilent technologies a reported secondquarter ...,in this article we discuss growth stocks to wa...,selfaware solutions making mass spectrometry m...,streamlined analysis of pfas compounds in wast...,santa clara calif june business wireagilent te...,agilent technologies a expanded its liquid chr...,santa clara calif june business wireagilent te...,...,sectorfocus blog the rules of the biotech game...,in this article we discuss worst performing bi...,when it comes to biotech stocks i usually like...,in this piece we will take a look at the bigge...,in a notable insider transaction evp chief pro...,its not a stretch to say that informatica incs...,infinera corporation nasdaqinfn q earnings cal...,infinera corporationsan jose calif nov globe n...,ing groupprogress on share buyback programmein...,bloomberg us borrowers are retreating en masse...
Unnamed: 0,5911,5912,5913,5915,5917,5921,5922,5923,5924,5925,...,7489,7491,7559,7560,518,519,4136,4141,5971,5971
publish_time,2023-05-18 00:00:00,2023-05-19 00:00:00,2023-05-22 00:00:00,2023-05-24 00:00:00,2023-05-26 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00,2023-06-06 00:00:00,2023-06-07 00:00:00,2023-06-08 00:00:00,...,2023-08-04 00:00:00,2023-08-08 00:00:00,2023-11-13 00:00:00,2023-11-14 00:00:00,2023-11-17 00:00:00,2023-11-20 00:00:00,2023-11-09 00:00:00,2023-11-16 00:00:00,2018-02-02 00:00:00,2018-02-02 00:00:00
Open,127.650002,129.589996,128.660004,115.199997,120.07,117.769997,118.580002,118.779999,117.169998,115.93,...,64.459999,65.269997,52.09,53.599998,24.42,24.459999,3.65,4.36,19.67,19.67
High,129.330002,130.470001,129.75,121.089996,120.720001,118.410004,119.760002,119.220001,117.370003,116.900002,...,65.099998,65.540001,53.41,54.810001,24.65,24.809999,4.23,4.36,19.700001,19.700001
Low,126.489998,128.619995,128.289993,113.279999,118.379997,116.019997,117.089996,116.639999,115.139999,114.660004,...,63.580002,64.260002,52.09,53.380001,24.25,24.261999,3.61,4.09,19.35,19.35
Close,129.139999,128.869995,129.089996,120.989998,120.419998,118.220001,118.349998,117.440002,116.589996,114.690002,...,63.630001,64.529999,53.119999,54.57,24.48,24.790001,3.77,4.22,19.35,19.35
Adj Close,129.139999,128.869995,129.089996,120.989998,120.419998,118.220001,118.349998,117.440002,116.589996,114.690002,...,63.630001,64.529999,53.119999,54.57,24.48,24.790001,3.77,4.22,13.294787,13.294787


In [ ]:
merged_df.to_csv('df2.csv')

In [ ]:
df_new = pd.read_csv('df2.csv')

In [ ]:
merged_df = df_new.T.drop_duplicates().reset_index(drop=True)
merged_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,symbol,publish_time,news_data_combined,Unnamed: 0,publish_time,Open,High,Low,Close,Adj Close,Volume,symbol
1,A,2023-05-18 00:00:00,vipshop holdings limited vips is set to report...,5911,2023-05-18 00:00:00,127.6500015258789,129.3300018310547,126.48999786376952,129.13999938964844,129.13999938964844,1637000.0,A
2,A,2023-05-19 00:00:00,santa clara calif may business wireagilent tec...,5912,2023-05-19 00:00:00,129.58999633789062,130.47000122070312,128.6199951171875,128.8699951171875,128.8699951171875,1480400.0,A
3,A,2023-05-21 00:00:00,earnings reports from zoom video lowes snowfla...,5913,2023-05-22 00:00:00,128.66000366210938,129.75,128.2899932861328,129.08999633789062,129.08999633789062,1764500.0,A
4,A,2023-05-24 00:00:00,agilent technologies a reported secondquarter ...,5915,2023-05-24 00:00:00,115.1999969482422,121.08999633789062,113.27999877929688,120.98999786376952,120.98999786376952,6850400.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
7092,INFA,2023-11-19 00:00:00,its not a stretch to say that informatica incs...,519,2023-11-20 00:00:00,24.459999084472656,24.809999465942383,24.261999130249023,24.790000915527344,24.790000915527344,563200.0,INFA
7093,INFN,2023-11-09 00:00:00,infinera corporation nasdaqinfn q earnings cal...,4136,2023-11-09 00:00:00,3.650000095367432,4.230000019073486,3.609999895095825,3.7699999809265137,3.7699999809265137,12329500.0,INFN
7094,INFN,2023-11-16 00:00:00,infinera corporationsan jose calif nov globe n...,4141,2023-11-16 00:00:00,4.360000133514404,4.360000133514404,4.090000152587891,4.21999979019165,4.21999979019165,2498700.0,INFN
7095,ING,2023-11-14 00:00:00,ing groupprogress on share buyback programmein...,5971,2018-02-02 00:00:00,19.670000076293945,19.700000762939453,19.350000381469727,19.350000381469727,13.294787406921388,2353000.0,ING


In [ ]:
# Set the first row as column names
merged_df.columns = merged_df.iloc[0]

# Drop the first row as it's now redundant
merged_df = merged_df.drop(0)

# Reset the index after dropping the first row
merged_df = merged_df.reset_index(drop=True)

merged_df

,symbol,publish_time,news_data_combined,Unnamed: 0,publish_time,Open,High,Low,Close,Adj Close,Volume,symbol
0,A,2023-05-18 00:00:00,vipshop holdings limited vips is set to report...,5911,2023-05-18 00:00:00,127.6500015258789,129.3300018310547,126.48999786376952,129.13999938964844,129.13999938964844,1637000.0,A
1,A,2023-05-19 00:00:00,santa clara calif may business wireagilent tec...,5912,2023-05-19 00:00:00,129.58999633789062,130.47000122070312,128.6199951171875,128.8699951171875,128.8699951171875,1480400.0,A
2,A,2023-05-21 00:00:00,earnings reports from zoom video lowes snowfla...,5913,2023-05-22 00:00:00,128.66000366210938,129.75,128.2899932861328,129.08999633789062,129.08999633789062,1764500.0,A
3,A,2023-05-24 00:00:00,agilent technologies a reported secondquarter ...,5915,2023-05-24 00:00:00,115.1999969482422,121.08999633789062,113.27999877929688,120.98999786376952,120.98999786376952,6850400.0,A
4,A,2023-05-26 00:00:00,in this article we discuss growth stocks to wa...,5917,2023-05-26 00:00:00,120.06999969482422,120.72000122070312,118.37999725341795,120.41999816894533,120.41999816894533,2089100.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
7091,INFA,2023-11-19 00:00:00,its not a stretch to say that informatica incs...,519,2023-11-20 00:00:00,24.459999084472656,24.809999465942383,24.261999130249023,24.790000915527344,24.790000915527344,563200.0,INFA
7092,INFN,2023-11-09 00:00:00,infinera corporation nasdaqinfn q earnings cal...,4136,2023-11-09 00:00:00,3.650000095367432,4.230000019073486,3.609999895095825,3.7699999809265137,3.7699999809265137,12329500.0,INFN
7093,INFN,2023-11-16 00:00:00,infinera corporationsan jose calif nov globe n...,4141,2023-11-16 00:00:00,4.360000133514404,4.360000133514404,4.090000152587891,4.21999979019165,4.21999979019165,2498700.0,INFN
7094,ING,2023-11-14 00:00:00,ing groupprogress on share buyback programmein...,5971,2018-02-02 00:00:00,19.670000076293945,19.700000762939453,19.350000381469727,19.350000381469727,13.294787406921388,2353000.0,ING


In [ ]:
# Define new column names
new_column_names = ['symbol', 'publish_time', 'news_data_combined', 'remove', 'pub_time', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Sy']

# Rename columns
merged_df.columns = new_column_names
merged_df

,symbol,publish_time,news_data_combined,remove,pub_time,Open,High,Low,Close,Adj Close,Volume,Sy
0,A,2023-05-18 00:00:00,vipshop holdings limited vips is set to report...,5911,2023-05-18 00:00:00,127.6500015258789,129.3300018310547,126.48999786376952,129.13999938964844,129.13999938964844,1637000.0,A
1,A,2023-05-19 00:00:00,santa clara calif may business wireagilent tec...,5912,2023-05-19 00:00:00,129.58999633789062,130.47000122070312,128.6199951171875,128.8699951171875,128.8699951171875,1480400.0,A
2,A,2023-05-21 00:00:00,earnings reports from zoom video lowes snowfla...,5913,2023-05-22 00:00:00,128.66000366210938,129.75,128.2899932861328,129.08999633789062,129.08999633789062,1764500.0,A
3,A,2023-05-24 00:00:00,agilent technologies a reported secondquarter ...,5915,2023-05-24 00:00:00,115.1999969482422,121.08999633789062,113.27999877929688,120.98999786376952,120.98999786376952,6850400.0,A
4,A,2023-05-26 00:00:00,in this article we discuss growth stocks to wa...,5917,2023-05-26 00:00:00,120.06999969482422,120.72000122070312,118.37999725341795,120.41999816894533,120.41999816894533,2089100.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
7091,INFA,2023-11-19 00:00:00,its not a stretch to say that informatica incs...,519,2023-11-20 00:00:00,24.459999084472656,24.809999465942383,24.261999130249023,24.790000915527344,24.790000915527344,563200.0,INFA
7092,INFN,2023-11-09 00:00:00,infinera corporation nasdaqinfn q earnings cal...,4136,2023-11-09 00:00:00,3.650000095367432,4.230000019073486,3.609999895095825,3.7699999809265137,3.7699999809265137,12329500.0,INFN
7093,INFN,2023-11-16 00:00:00,infinera corporationsan jose calif nov globe n...,4141,2023-11-16 00:00:00,4.360000133514404,4.360000133514404,4.090000152587891,4.21999979019165,4.21999979019165,2498700.0,INFN
7094,ING,2023-11-14 00:00:00,ing groupprogress on share buyback programmein...,5971,2018-02-02 00:00:00,19.670000076293945,19.700000762939453,19.350000381469727,19.350000381469727,13.294787406921388,2353000.0,ING


In [ ]:
# Print the column names with their corresponding indices
for index, column_name in enumerate(merged_df.columns):
    print(f"Column '{column_name}' has index {index}")

Column 'symbol' has index 0
Column 'publish_time' has index 1
Column 'news_data_combined' has index 2
Column 'remove' has index 3
Column 'pub_time' has index 4
Column 'Open' has index 5
Column 'High' has index 6
Column 'Low' has index 7
Column 'Close' has index 8
Column 'Adj Close' has index 9
Column 'Volume' has index 10
Column 'Sy' has index 11


In [ ]:
# Define the indices of columns to drop
columns_to_drop = [3, 4, 11]

# Drop columns by index
merged_df = merged_df.drop(merged_df.columns[columns_to_drop], axis=1)

merged_df

,symbol,publish_time,news_data_combined,Open,High,Low,Close,Adj Close,Volume
0,A,2023-05-18 00:00:00,vipshop holdings limited vips is set to report...,127.6500015258789,129.3300018310547,126.48999786376952,129.13999938964844,129.13999938964844,1637000.0
1,A,2023-05-19 00:00:00,santa clara calif may business wireagilent tec...,129.58999633789062,130.47000122070312,128.6199951171875,128.8699951171875,128.8699951171875,1480400.0
2,A,2023-05-21 00:00:00,earnings reports from zoom video lowes snowfla...,128.66000366210938,129.75,128.2899932861328,129.08999633789062,129.08999633789062,1764500.0
3,A,2023-05-24 00:00:00,agilent technologies a reported secondquarter ...,115.1999969482422,121.08999633789062,113.27999877929688,120.98999786376952,120.98999786376952,6850400.0
4,A,2023-05-26 00:00:00,in this article we discuss growth stocks to wa...,120.06999969482422,120.72000122070312,118.37999725341795,120.41999816894533,120.41999816894533,2089100.0
...,...,...,...,...,...,...,...,...,...
7091,INFA,2023-11-19 00:00:00,its not a stretch to say that informatica incs...,24.459999084472656,24.809999465942383,24.261999130249023,24.790000915527344,24.790000915527344,563200.0
7092,INFN,2023-11-09 00:00:00,infinera corporation nasdaqinfn q earnings cal...,3.650000095367432,4.230000019073486,3.609999895095825,3.7699999809265137,3.7699999809265137,12329500.0
7093,INFN,2023-11-16 00:00:00,infinera corporationsan jose calif nov globe n...,4.360000133514404,4.360000133514404,4.090000152587891,4.21999979019165,4.21999979019165,2498700.0
7094,ING,2023-11-14 00:00:00,ing groupprogress on share buyback programmein...,19.670000076293945,19.700000762939453,19.350000381469727,19.350000381469727,13.294787406921388,2353000.0


In [ ]:
merged_df['publish_time'] = pd.to_datetime(merged_df['publish_time'])
merged_df['publish_time'] = merged_df['publish_time'].dt.date
merged_df

,symbol,publish_time,news_data_combined,Open,High,Low,Close,Adj Close,Volume
0,A,2023-05-18,vipshop holdings limited vips is set to report...,127.6500015258789,129.3300018310547,126.48999786376952,129.13999938964844,129.13999938964844,1637000.0
1,A,2023-05-19,santa clara calif may business wireagilent tec...,129.58999633789062,130.47000122070312,128.6199951171875,128.8699951171875,128.8699951171875,1480400.0
2,A,2023-05-21,earnings reports from zoom video lowes snowfla...,128.66000366210938,129.75,128.2899932861328,129.08999633789062,129.08999633789062,1764500.0
3,A,2023-05-24,agilent technologies a reported secondquarter ...,115.1999969482422,121.08999633789062,113.27999877929688,120.98999786376952,120.98999786376952,6850400.0
4,A,2023-05-26,in this article we discuss growth stocks to wa...,120.06999969482422,120.72000122070312,118.37999725341795,120.41999816894533,120.41999816894533,2089100.0
...,...,...,...,...,...,...,...,...,...
7091,INFA,2023-11-19,its not a stretch to say that informatica incs...,24.459999084472656,24.809999465942383,24.261999130249023,24.790000915527344,24.790000915527344,563200.0
7092,INFN,2023-11-09,infinera corporation nasdaqinfn q earnings cal...,3.650000095367432,4.230000019073486,3.609999895095825,3.7699999809265137,3.7699999809265137,12329500.0
7093,INFN,2023-11-16,infinera corporationsan jose calif nov globe n...,4.360000133514404,4.360000133514404,4.090000152587891,4.21999979019165,4.21999979019165,2498700.0
7094,ING,2023-11-14,ing groupprogress on share buyback programmein...,19.670000076293945,19.700000762939453,19.350000381469727,19.350000381469727,13.294787406921388,2353000.0


In [ ]:
merged_df.to_csv('final2.csv')

# **Lets Calculate the stock Rise/Fall According to the news**

Here we will be analyzing all the news data to predict sentiments using VADER Library.

In [ ]:
df_vader = pd.read_csv('final2.csv')

In [ ]:
df_vader = df_vader.drop(['Unnamed: 0'], axis=1)
df_vader

,symbol,publish_time,news_data_combined,Open,High,Low,Close,Adj Close,Volume
0,A,2023-05-18,vipshop holdings limited vips is set to report...,127.650002,129.330002,126.489998,129.139999,129.139999,1637000.0
1,A,2023-05-19,santa clara calif may business wireagilent tec...,129.589996,130.470001,128.619995,128.869995,128.869995,1480400.0
2,A,2023-05-21,earnings reports from zoom video lowes snowfla...,128.660004,129.750000,128.289993,129.089996,129.089996,1764500.0
3,A,2023-05-24,agilent technologies a reported secondquarter ...,115.199997,121.089996,113.279999,120.989998,120.989998,6850400.0
4,A,2023-05-26,in this article we discuss growth stocks to wa...,120.070000,120.720001,118.379997,120.419998,120.419998,2089100.0
...,...,...,...,...,...,...,...,...,...
7091,INFA,2023-11-19,its not a stretch to say that informatica incs...,24.459999,24.809999,24.261999,24.790001,24.790001,563200.0
7092,INFN,2023-11-09,infinera corporation nasdaqinfn q earnings cal...,3.650000,4.230000,3.610000,3.770000,3.770000,12329500.0
7093,INFN,2023-11-16,infinera corporationsan jose calif nov globe n...,4.360000,4.360000,4.090000,4.220000,4.220000,2498700.0
7094,ING,2023-11-14,ing groupprogress on share buyback programmein...,19.670000,19.700001,19.350000,19.350000,13.294787,2353000.0


In [ ]:
new_words =  {'falls': -9, 'drops': -9, 'rise': 9, 'increases': 9, 'gain': 9, 'hiked': -9, 'dips': -9, 'declines': -9, 'decline': -9, 'hikes': -9, 'jumps': 9,
              'lose': -9, 'profit': 9, 'loss': -9, 'shreds': -9, 'sell': -9, 'buy': 9, 'recession': -9, 'rupee weakens': -9, 'record low': -9, 'record high': 9,
              'sensex up': 9, 'nifty down': -9, 'sensex down': -9, 'nifty up': 9}

analyser = SentimentIntensityAnalyzer()
analyser.lexicon.update(new_words)

for i in tqdm(df_vader.itertuples()):
    score = analyser.polarity_scores(df_vader.iloc[i[0]]['news_data_combined'])

    df_vader.at[i[0], 'score'] = score['compound']

    if score['compound'] >= 0:
        df_vader.at[i[0], 'sentiment'] = 1
    else:
        df_vader.at[i[0], 'sentiment'] = -1

7096it [20:50,  5.67it/s]


In [ ]:
df_vader.to_csv("sentiment.csv")